# Dynamics tutorial

## Packages

We will use the `contracts` and `six` libraries to create the classes,
as well as all previously introduced libraries.


In [142]:
from abc import ABCMeta, abstractmethod

from contracts import contract, check_isinstance
from six import with_metaclass

from duckietown_serialization_ds1 import Serializable

import geometry as geo
import numpy as np

# from generic_kinematics.py
from geometry.poses import *

# Abstract classes

In `duckietown-world` there is an interface for the dynamics of a platform. The interface is:

In [143]:
class PlatformDynamics(with_metaclass(ABCMeta)):
    """
        This class represents the state of a dynamical system.


            s0 = ...

            s1 = s0.integrate(dt=0.1, commands=[1,1])
            s2 = s0.integrate(dt=0.1, commands=[1,1])


        Each subclass has its own representation of commands.

    """

    @abstractmethod
    @contract(dt='>0')
    def integrate(self, dt, commands):
        """
            Returns the result of applying commands for dt.

            :param dt: time interval
            :param commands: class-specific commands
            :return: the next state
        """

    @abstractmethod
    @contract(returns='TSE2')
    def TSE2_from_state(self):
        """ Returns pose, velocity for the state. """


class PlatformDynamicsFactory(with_metaclass(ABCMeta)):

    @classmethod
    @abstractmethod
    @contract(c0='TSE2', returns=PlatformDynamics)
    def initialize(cls, c0, t0=0, seed=None):
        """
            Returns the dynamics initalized at a certain configuration.

            :param c0: configuration in TSE2
            :param t0: time in which to sstart
            :param seed: seed for a possible random number generator

        """


## Generic Kinematics 

In [144]:
class GenericKinematicsSE2(PlatformDynamicsFactory, PlatformDynamics, Serializable):
    """
        Any dynamics on SE(2)

        Commands = velocities in se(2)
    """

    @classmethod
    @contract(c0='TSE2')
    def initialize(cls, c0, t0=0, seed=None):
        return GenericKinematicsSE2(c0, t0)

    @contract(c0='TSE2')
    def __init__(self, c0, t0):
        # start at q0, v0
        q0, v0 = c0
        geo.SE2.belongs(q0)
        geo.se2.belongs(v0)
        self.t0 = t0
        self.v0 = v0
        self.q0 = q0

    def integrate(self, dt, commands):
        """ commands = velocity in body frame """
        # convert to float
        dt = float(dt)
        # the commands must belong to se(2)
        geo.se2.belongs(commands)
        v = commands
        # suppose we hold v for dt, which pose are we going to?
        diff = geo.SE2.group_from_algebra(dt * v) # exponential map
        # compute the absolute new pose; applying diff from q0
        q1 = geo.SE2.multiply(self.q0, diff)
        # the new configuration
        c1 = q1, v
        # the new time
        t1 = self.t0 + dt
        # return the new state
        return GenericKinematicsSE2(c1, t1)

    @contract(returns='TSE2')
    def TSE2_from_state(self):
        return self.q0, self.v0

DEBUG:dt-serialization:Registering class GenericKinematicsSE2


### Model Parameters

In [145]:
class DynamicModelParameters(PlatformDynamicsFactory, Serializable):
    '''
        This class represents the parameters of the ideal differential drive dynamics.

        radius_left, radius_right: wheels radii
        wheel_distance: distance between two wheels

    '''

    def __init__(self, u1, u2, u3, w1, w2, w3, uar, ual, war, wal):
        # parameters for autonomous dynamics
        self.u1 = u1
        self.u2 = u2
        self.u3 = u3
        self.w1 = w1
        self.w2 = w2
        self.w3 = w3

        # parameters for forced dynamics
        self.u_alpha_r = uar
        self.u_alpha_l = ual
        self.w_alpha_r = war
        self.w_alpha_l = wal

    def initialize(self, c0, t0=0, seed=None):
        return DifferentialDriveDynamics(self, c0, t0)

DEBUG:dt-serialization:Registering class DynamicModelParameters


### Wheel Commands

In [146]:
class WheelVelocityCommands(Serializable):
    def __init__(self, left_wheel_angular_velocity, right_wheel_angular_velocity):
        self.left_wheel_angular_velocity = left_wheel_angular_velocity
        self.right_wheel_angular_velocity = right_wheel_angular_velocity

DEBUG:dt-serialization:Registering class WheelVelocityCommands


### Differential Drive Dynamics

In [147]:
class DynamicModel(GenericKinematicsSE2):
    """
        This represents a dynamical formulation of of a differential-drive vehicle.
    """

    def __init__(self, parameters, c0, t0):
        """
        :param parameters:  instance of DifferentialDriveDynamicsParameters
        :param c0: initial configuration
        :param t0: initial time
        """
        check_isinstance(parameters, DynamicModelParameters)
        self.parameters = parameters
        GenericKinematicsSE2.__init__(self, c0, t0)

    @staticmethod
    def model(input, parameters, u=None, w=None):
        ## Unpack Inputs
        U = np.array([input.right_wheel_angular_velocity, input.left_wheel_angular_velocity])
        V = U.reshape(U.size, 1)

        ## Unpack Parameters
        check_isinstance(parameters, DynamicModelParameters)
        
        # parameters for autonomous dynamics
        u1 = parameters.u1
        u2 = parameters.u2
        u3 = parameters.u3
        w1 = parameters.w1
        w2 = parameters.w2
        w3 = parameters.w3
        # parameters for forced dynamics
        u_alpha_r = parameters.u_alpha_r
        u_alpha_l = parameters.u_alpha_l
        w_alpha_r = parameters.w_alpha_r
        w_alpha_l = parameters.w_alpha_l

        ## Calculate Dynamics
        # Nonlinear Dynamics - autonomous response
        f_dynamic = np.array([
            [-u1 * u - u2 * w + u3 * w ** 2],
            [-w1 * w - w2 * u - w3 * u * w]
        ])

        # Input Matrix
        B = np.array([
            [u_alpha_r, u_alpha_l],
            [w_alpha_r, -w_alpha_l]
        ])
        # Forced response
        f_forced = np.matmul(B, V)

        # acceleration
        x_dot_dot = f_dynamic + f_forced

        return x_dot_dot
    
    def integrate(self, dt, commands):
        """

        :param dt:
        :param commands: an instance of WheelVelocityCommands
        :return:
        """
        check_isinstance(commands, WheelVelocityCommands)
        
        # previous velocities 
        linear_angular_prev = linear_angular_from_se2(self.v0)
        linear_prev = linear_angular_prev[0]
        longit_prev = linear_prev[0]
        lateral_prev = linear_prev[1]
        angular_prev = linear_angular_prev[1]
        
        x_dot_dot = self.model(commands, parameters, u=longit_prev, w=angular_prev)
        
        ## first convert acceleration to velocity by forward euler
        longitudinal = longit_prev + dt * x_dot_dot[0]
        angular = angular_prev + dt * x_dot_dot[1]
        lateral = 0.0
        
        linear = [longitudinal, lateral]
        
        # represent this as se(2)
        commands_se2 = geo.se2_from_linear_angular(linear, angular)

        # Call the "integrate" function of GenericKinematicsSE2
        s1 = GenericKinematicsSE2.integrate(self, dt, commands_se2)

        # new state
        c1 = s1.q0, s1.v0
        t1 = s1.t0
        
        return DynamicModel(self.parameters, c1, t1)
    

DEBUG:dt-serialization:Registering class DynamicModel


### Simulation

In [148]:
# Model Parameters
u2 = u3 = w1 = w2 = w3 = 0 # to simplify the model
u1 = w1 = 1 # main contributor from unforced dynamics
uar = ual = war = wal = 1 # input matrix

parameters = DynamicModelParameters(u1, u2, u3, w1, w2, w3, uar, ual, war, wal)

# initial configuration
init_pose = np.array([0,0])
init_vel = np.array([0,0])
init_time = 0

q0 = geo.SE2_from_R2(init_pose)
v0 = geo.se2_from_linear_angular(init_vel, 0)
c0 = q0, v0

# starting time
t0 = 0

system = DynamicModel(parameters, c0, t0)

del_t = 1

for i in range(5):
    # input = (left: -0.1, right: 0.1) must generate a counter-clockwise rotation (increasing theta)
    myCarCommands = WheelVelocityCommands(-0.1, 0.1)
    system = system.integrate(del_t, myCarCommands)
    
    current_p, theta = geo.translation_angle_from_SE2(system.q0)
    
    print('pose: {}'.format(current_p))
    print('theta: {}'.format(np.rad2deg(theta)))
    

pose: [0. 0.]
theta: 11.4591559026
pose: [0. 0.]
theta: 22.9183118052
pose: [0. 0.]
theta: 34.3774677078
pose: [0. 0.]
theta: 45.8366236105
pose: [0. 0.]
theta: 57.2957795131
